In [10]:
!pip install kaggle torch torchvision transformers matplotlib safetensors


In [11]:
from google.colab import files
files.upload()   # upload kaggle.json


Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"prateekshetty28","key":"253cf17c53d9ab0465b6311ee47cdce7"}'}

In [12]:
!kaggle datasets download -d jonathanoheix/face-expression-recognition-dataset


Dataset URL: https://www.kaggle.com/datasets/jonathanoheix/face-expression-recognition-dataset
License(s): unknown
 91% 110M/121M [00:00<00:00, 181MB/s] 
100% 121M/121M [00:00<00:00, 178MB/s]


In [13]:
import zipfile

with zipfile.ZipFile("face-expression-recognition-dataset.zip", "r") as zip_ref:
    zip_ref.extractall("dataset")

print("Extracted dataset.")


Extracted dataset.


In [14]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

train_dir = "dataset/images/"

# Train transforms
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

dataset = datasets.ImageFolder(train_dir, transform=train_transforms)

train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

print("Classes:", dataset.classes)


Classes: ['images', 'train', 'validation']


In [15]:
from transformers import ViTForImageClassification, ViTConfig
import torch

num_classes = 7  # angry, disgust, fear, happy, sad, surprise, neutral

config = ViTConfig.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=num_classes
)

model = ViTForImageClassification(config)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print("Model ready on:", device)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

Model ready on: cpu


In [16]:
import torch.optim as optim
from torch.nn import CrossEntropyLoss

criterion = CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=3e-5)
epochs = 15   # increase if needed


In [ ]:
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images).logits
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")


In [ ]:
save_path = "emotion_model.pt"
torch.save(model.state_dict(), save_path)

print("Saved:", save_path)


In [ ]:
from google.colab import files
files.download("emotion_model.pt")
